<a href="https://colab.research.google.com/github/FilipeMSouza/ml-2023-1-trabalho-final/blob/master/Trabalho%20Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_train = pd.read_csv("train.csv")

In [ ]:
df_train.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,Bruno Mars,That's What I Like (feat. Gucci Mane),60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,NaN,0.0849,0.8990,134.071,234596.0,4,5
1,Boston,Hitch a Ride,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,The Raincoats,No Side to Fall In,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,Deno,Lingo (feat. J.I & Chunkz),66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,NaN,0.1220,0.5690,107.033,173968.0,4,5
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


In [ ]:
for col in df_train.columns[df_train.isna().any()].tolist():
    print(col + ": " + str(df_train[col].nunique()))

Popularity: 100
key: 11
instrumentalness: 4253


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_train["key"].mode()

0    7.0
Name: key, dtype: float64

In [ ]:
mode = df_train["Popularity"].mode()
df_train["Popularity"] = df_train["Popularity"].fillna(mode[0])
mode = df_train["key"].mode()
df_train["key"] = df_train["key"].fillna(mode[0])
median = df_train["instrumentalness"].median()
df_train["instrumentalness"] =  df_train["instrumentalness"].fillna(median)

In [ ]:
df_train.isnull().sum()

Artist Name           0
Track Name            0
Popularity            0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_in min/ms    0
time_signature        0
Class                 0
dtype: int64

In [ ]:
df_train["Popularity"] = pd.cut(df_train["Popularity"], bins=10, labels=False)
df_test["Popularity"] = pd.cut(df_test["Popularity"], 10, labels=False)

In [ ]:
df_train[["key", "mode", "time_signature", "Popularity"]] = df_train[["key", "mode", "time_signature", "Popularity"]].astype("category")
df_test[["key", "mode", "time_signature", "Popularity"]] = df_test[["key", "mode", "time_signature", "Popularity"]].astype("category")

In [ ]:
Y = df_train["Class"]
X = df_train.drop(["Class", "Artist Name", "Track Name"], axis=1)
cat_cols = X.select_dtypes("category").columns
num_cols = X.select_dtypes(include=np.number).columns
print(cat_cols)
print(num_cols)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

Index(['Popularity', 'key', 'mode', 'time_signature'], dtype='object')
Index(['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_in min/ms'],
      dtype='object')


In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train[num_cols])
x_test_scaled = scaler.transform(x_test[num_cols])

In [ ]:
scaled_features_df_train = pd.DataFrame(x_train_scaled, index=x_train[num_cols].index, columns=x_train[num_cols].columns)
scaled_features_df_test = pd.DataFrame(x_test_scaled, index=x_test[num_cols].index, columns=x_test[num_cols].columns)
df_train_prepro = pd.concat((scaled_features_df_train, x_train[cat_cols]), axis=1)
df_test_prepro = pd.concat((scaled_features_df_test, x_test[cat_cols]), axis=1)

In [ ]:
modelos = [
    ("log_reg", LogisticRegression(multi_class="multinomial", max_iter=1000)),
    ("naive_bay", GaussianNB()),
    ("kn_clf", KNeighborsClassifier()),
    ("dtree", DecisionTreeClassifier()),
    ("rforest", RandomForestClassifier()),
    ]

In [ ]:
df_metrics = pd.DataFrame(index=["name", "accuracy", "precision", "recall", "f1_score"])
n=0
for name, model in modelos:
    model.fit(df_train_prepro, y_train)
    y_pred = model.predict(df_test_prepro)
    metrics = [name]
    metrics.append(accuracy_score(y_test, y_pred))
    metrics.append(precision_score(y_test, y_pred, average="weighted"))
    metrics.append(recall_score(y_test, y_pred, average="weighted"))
    metrics.append(f1_score(y_test, y_pred, average="weighted"))
    df_metrics[n] = metrics
    n += 1

CPU times: user 16.2 s, sys: 5.96 s, total: 22.2 s
Wall time: 14.9 s


In [ ]:
display(df_metrics)

,0,1,2,3,4
name,log_reg,naive_bay,kn_clf,dtree,rforest
accuracy,0.488333,0.445,0.416111,0.361389,0.497222
precision,0.457426,0.41921,0.412866,0.375583,0.483663
recall,0.488333,0.445,0.416111,0.361389,0.497222
f1_score,0.460276,0.420561,0.413992,0.367805,0.484367
